In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm



In [14]:
# Define custom dataset class
class FireDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_name)
        label_path = os.path.join(self.label_dir, image_name.replace('.jpg', '.txt'))

        # Load image
        image = Image.open(image_path).convert('RGB')

        # Determine label based on label file content
        if os.path.exists(label_path) and os.stat(label_path).st_size > 0:
            label = 1  # Fire
        else:
            label = 0  # No Fire

        if self.transform:
            image = self.transform(image)

        return image, label

In [15]:
# Directories
image_dir = '../dfire_train/images'
label_dir = '../dfire_train/labels'

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset and split into train and validation sets
dataset = FireDataset(image_dir, label_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [16]:
# Load ResNet50 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classification
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [17]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update metrics
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / total
        train_accuracy = correct / total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= val_total
        val_accuracy = val_correct / val_total

        # Print metrics
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        print('-' * 50)


In [18]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch 1/20 - Validation: 100%|██████████| 216/216 [00:28<00:00,  7.71it/s]


Epoch 1/20
Training Loss: 0.5676, Training Accuracy: 0.7001
Validation Loss: 0.5044, Validation Accuracy: 0.7283
--------------------------------------------------


Epoch 2/20 - Validation: 100%|██████████| 216/216 [00:28<00:00,  7.66it/s]


Epoch 2/20
Training Loss: 0.4461, Training Accuracy: 0.7922
Validation Loss: 0.3774, Validation Accuracy: 0.8418
--------------------------------------------------


Epoch 3/20 - Validation: 100%|██████████| 216/216 [00:31<00:00,  6.84it/s]


...

Epoch 18/20
Training Loss: 0.0671, Training Accuracy: 0.9763
Validation Loss: 0.2847, Validation Accuracy: 0.9089
--------------------------------------------------


Epoch 19/20 - Validation: 100%|██████████| 216/216 [00:27<00:00,  7.80it/s]


Epoch 19/20
Training Loss: 0.0611, Training Accuracy: 0.9784
Validation Loss: 0.2599, Validation Accuracy: 0.9205
--------------------------------------------------


Epoch 20/20 - Validation: 100%|██████████| 216/216 [00:27<00:00,  7.74it/s]

Epoch 20/20
Training Loss: 0.0565, Training Accuracy: 0.9803
Validation Loss: 0.2840, Validation Accuracy: 0.9083
--------------------------------------------------


In [19]:
# Save the trained model
torch.save(model.state_dict(), "dfire_resnet50.pth")
# print("Model saved as 'deepfire_resnet50.pth'")